# Ungraded Lab - Gradient for regularized logistic regression


## Goals
In this lab you will:
- extend the implementation of determining the gradient to include regularization.
- write a version using looping
- optionally write a vectorized version
- speed test the difference


In [ ]:
import numpy as np
from lab_utils import sigmoid

## Looping Version

The gradient of the regularized cost function the partial deriviative of cost relative to the parameters $w$ and $b$:

$$\frac{\partial J(\mathbf{w})}{\partial b} = \frac{1}{m}  \sum_{i=0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})  \tag {1}$$

$$\frac{\partial J(\mathbf{w},b)}{\partial w_j} = \left( \frac{1}{m}  \sum_{i=0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) x_j^{(i)} \right) +  \frac{\lambda}{m} w_j  \quad\, \mbox{for $j=0...(n-1)$} \tag {2}$$


You will implement a function called `compute_gradient_reg` which will return $\frac{\partial J(\mathbf{w},b)}{\partial w},\frac{\partial J(\mathbf{w},b)}{\partial b}$.

Please complete the `compute_gradient_reg` function:

- Calculate the gradient for each element `dJdw` and `dJdb` exactly as you did in the `compute_gradient` function in earlier labs:
    - initialize variables to accumulate `dJdw` and `dJdb`
    - loop over all examples
        - calculate the error for that example $g(\mathbf{x}^{(i)T}\mathbf{w} + b) - \mathbf{y}^{(i)}$
        - add the error to `dJdb` (equation 1 above)
        - for each input value $x_{j}^{(i)}$ in this example,  
            - multiply the error by the input  $x_{j}^{(i)}$, and add to the corresponding element of `dJdw`. (equation 2 above)
     - divide `dJdb` and `dJdw` by total number of examples (m)
- Now compute the regularization term
    - loop over all $w$
        - add  $\frac{\lambda}{m} * w_j$ to the corresponding element of  `dJdw`

As you are doing this, remember that the variables X and y are not scalar values but matrices of shape ($m, n$) and ($𝑚$,1 ) respectively, where  $𝑛$ is the number of features and $𝑚$ is the number of training examples. 


<details>
  <summary><font size="2" color="darkgreen"><b>Hints</b></font></summary>
    
```python     
def compute_gradient_reg(X, y, w, b, lambda_ = 1): 
    """
    Computes the gradient for linear regression 
 
    Args:
      X : (array_like Shape (m,n)) variable such as house size 
      y : (array_like Shape (m,1)) actual value 
      w : (array_like Shape (n,1)) values of parameters of the model      
      b : (scalar)                 value of parameter of the model  
      lambda_ : (scalar,float)      regularization constant
    Returns
      dJdw: (array_like Shape (n,1)) The gradient of the cost w.r.t. the parameters w. 
      dJdb: (scalar)                The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape
    dJdw = np.zeros((n,1))
    dJdb = 0.
    err  = 0.

    ### START CODE HERE ### 

    for i in range(m):
        err = sigmoid(X[i] @ w + b)  - y[i]    
        for j in range(n):
            dJdw[j] = dJdw[j] + err * X[i][j]
        dJdb = dJdb + err
    dJdw = dJdw/m
    dJdb = dJdb/m
    
    for j in range(n):
        dJdw[j] = dJdw[j] + (lambda_/m) * w[j]


    ### END CODE HERE ###         
        
    return dJdb[0],dJdw  #index dJdb to return scalar value
```
</details>

In [ ]:
def compute_gradient_reg(X, y, w, b, lambda_ = 1): 
    """
    Computes the gradient for linear regression 
 
    Args:
      X : (array_like Shape (m,n)) variable such as house size 
      y : (array_like Shape (m,1)) actual value 
      w : (array_like Shape (n,1)) values of parameters of the model      
      b : (scalar)                 value of parameter of the model  
      lambda_ : (scalar,float)      regularization constant
    Returns
      dJdw: (array_like Shape (n,1)) The gradient of the cost w.r.t. the parameters w. 
      dJdb: (scalar)                The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape
    dJdw = np.zeros((n,1))
    dJdb = 0.
    err  = 0.

    ### START CODE HERE ### 
    ### BEGIN SOLUTION ###
    for i in range(m):
        err = sigmoid(X[i] @ w + b)  - y[i]    
        for j in range(n):
            dJdw[j] = dJdw[j] + err * X[i][j]
        dJdb = dJdb + err
    dJdw = dJdw/m
    dJdb = dJdb/m
    
    for j in range(n):
        dJdw[j] = dJdw[j] + (lambda_/m) * w[j]

    ### END SOLUTION ### 
    ### END CODE HERE ###         
        
    return dJdb[0],dJdw  #index dJdb to return scalar value

Run the cell below to check your implementation of the `compute_gradient_reg` function.

In [ ]:
np.random.seed(1)
X_tmp = np.random.rand(5,3)
y_tmp = np.array([0,1,0,1,0]).reshape(-1,1)
initial_w  = np.random.rand(X_tmp.shape[1]).reshape(-1,1)
initial_b = 0.5
lambda_ = 1
dJdb, dJdw =  compute_gradient_reg(X_tmp, y_tmp, initial_w, initial_b, lambda_)

print(f"dJdb: {dJdb}", )
print(f"Regularized dJdw:\n {dJdw.tolist()}", )

**Expected Output**
```
dJdb: 0.341798994972791
Regularized dJdw:
 [[0.2140281799506471], [0.34511336695769707], [0.1412845236752601]]
 ```

## Vectorized Version

Using the equations above and previous experience with vectorized gradient descent in a Lab04, write `compute_gradient_reg_matrix`.

The first section is the same as in Lab 4. Add a final section which adds $\frac{\lambda}{m} * w_j$ to the corresponding element of $djdw$.

<details>
  <summary><font size="2" color="darkgreen"><b>Hints</b></font></summary>
    
```python     
def compute_gradient_reg_matrix(X, y, w, b, lambda_= 1): 
    """
    Computes the gradient for linear regression 
 
    Args:
      X : (array_like Shape (m,n)) variable such as house size 
      y : (array_like Shape (m,1)) actual value 
      w : (array_like Shape (n,1)) Values of parameters of the model      
      b : (scalar )                Values of parameter of the model      
      predict_function: (function) function to call to make prediction
    Returns
      dJdw: (array_like Shape (n,1)) The gradient of the cost w.r.t. the parameters w. 
      dJdb: (scalar)                The gradient of the cost w.r.t. the parameter b. 
                                  
    """
    m,n = X.shape
    ### START CODE HERE ### 

    f_wb = sigmoid(X @ w + b)      
    e   = f_wb - y                 
    dJdw  = (1/m) * (X.T @ e)      
    dJdb  = (1/m) * np.sum(e)  
    dJdw += (lambda_/m) * w

    ### END CODE HERE ###         
   
    return dJdb,dJdw
```
</details>

In [ ]:
def compute_gradient_reg_matrix(X, y, w, b, lambda_= 1): 
    """
    Computes the gradient for linear regression 
 
    Args:
      X : (array_like Shape (m,n)) variable such as house size 
      y : (array_like Shape (m,1)) actual value 
      w : (array_like Shape (n,1)) Values of parameters of the model      
      b : (scalar )                Values of parameter of the model      
      predict_function: (function) function to call to make prediction
    Returns
      dJdw: (array_like Shape (n,1)) The gradient of the cost w.r.t. the parameters w. 
      dJdb: (scalar)                The gradient of the cost w.r.t. the parameter b. 
                                  
    """
    m,n = X.shape
    ### START CODE HERE ### 
    ### BEGIN SOLUTION ###
    f_wb = sigmoid(X @ w + b)      
    e   = f_wb - y                 
    dJdw  = (1/m) * (X.T @ e)      
    dJdb  = (1/m) * np.sum(e)  
    dJdw += (lambda_/m) * w
    ### END SOLUTION ### 
    ### END CODE HERE ###         
   
    return dJdb,dJdw

In [ ]:
np.random.seed(1)
X_tmp = np.random.rand(5,3)
y_tmp = np.array([0,1,0,1,0]).reshape(-1,1)
initial_w  = np.random.rand(X_tmp.shape[1]).reshape(-1,1)
initial_b = 0.5
lambda_ = 1
dJdb, dJdw =  compute_gradient_reg_matrix(X_tmp, y_tmp, initial_w, initial_b, lambda_)

print(f"dJdb: {dJdb}", )
print(f"Regularized dJdw:\n {dJdw.tolist()}", )

**Expected Output**
```
dJdb: 0.34179899497279104
Regularized dJdw:
 [[0.2140281799506471], [0.3451133669576971], [0.14128452367526012]]
 ```

## Speed Test your implementations

In [ ]:
import time

np.random.seed(1)
m = 500; n=100
X_tmp = np.random.rand(m,n)
y_tmp = np.zeros((m,1))
initial_w  = np.random.rand(X_tmp.shape[1]).reshape(-1,1)
initial_b = 0.5
lambda_ = 1

start = time.time()
for i in range(10):
    dJdb, dJdw =  compute_gradient_reg(X_tmp, y_tmp, initial_w, initial_b, lambda_)
end = time.time()
print(f"non- matrix time {end - start}")


start = time.time()
for i in range(10):
    dJdb, dJdw =  compute_gradient_reg_matrix(X_tmp, y_tmp, initial_w, initial_b, lambda_)
end = time.time()
print(f"matrix time      {end - start}")


# 